In [ ]:
import pandas as pd
import os

In [ ]:
df_path = "data/NU_340297544_01AGO2023_31AGO2023.csv"
df = pd.read_csv(df_path, sep=",", encoding="latin-1")
df = df.rename({'DescriÃÂ§ÃÂ£o': 'Descricao'}, axis = 1)
df.head()

In [ ]:
def get_api_key():
    with open('API_KEY.txt', 'r') as file:
        api_key = file.read().strip()
    return api_key

In [ ]:
import os
import openai

def get_ai_response(descricao):
  descricao = 'Compra no dÃÂ©bito - Dellicata Paes e Tenta'

  openai.api_key = get_api_key() 
  resultado = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt="I'll give you a description in portugues-br, and you will give me one of this categorys that describes it 'Alimentacao', 'Pix', 'Outros', 'Compras online', 'Pagar Fatura'. Description = " + descricao,
    max_tokens=7,
    temperature=0
  )



  resposta = resultado["choices"][0]["text"]
  return resposta

In [ ]:
categorias = []
for linha in df['Descricao']:
    linha = linha.lower()
    categoria = get_ai_response(linha)
    categorias.append(categoria)
df['Categoria'] = categorias

In [ ]:
df